In [1]:
import numpy as np
import pandas as pd

In [3]:
d = {'Clubes': [], 'PG': [], 'j': [], 'V': [], 'E': [], 'D': [], 'GP': [], 'GC': [], 'SG': []}
dfClassChampionship = pd.DataFrame(data=d);
dfDataChampionship = pd.read_excel("b2006.xlsx", encoding='utf8')

In [4]:
dfDataChampionship.head()

,Horario,Dia,Data,Clube 1,Clube 2,Placar,Vencedor,Rodada,Arena
0,18h10,Sábado,15/04/2006,Juventude,Paraná,1x0,Juventude,1ª Rodada,Alfredo Jaconi
1,18h10,Sábado,15/04/2006,Vasco,Internacional,1x1,-,1ª Rodada,São Januário
2,16h00,Domingo,16/04/2006,São Paulo,Flamengo,1x0,São Paulo,1ª Rodada,Morumbi
3,16h00,Domingo,16/04/2006,Grêmio,Corinthians,2x0,Grêmio,1ª Rodada,Olímpico
4,16h00,Domingo,16/04/2006,Botafogo,Fortaleza,1x0,Botafogo,1ª Rodada,Maracanã


** Get all clubes for championship **



In [5]:
#Cancatenando as colunas Clube 1 e Clube 2
clubs  =  pd.concat([dfDataChampionship['Clube 1'].str.lower(), dfDataChampionship['Clube 2'].str.lower()], axis=1, keys=['Clubes'])

In [6]:
#Obtem todos os clubes 
cb = pd.DataFrame({'Clubes': clubs['Clubes'].unique()})

In [7]:
#Todos os clubes que participaram dp campeonato
cb

,Clubes
0,juventude
1,vasco
2,são paulo
3,grêmio
4,botafogo
5,são caetano
6,palmeiras
7,santa cruz
8,atlético-pr
9,goiás


In [12]:
def getNumberWinner(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter = df["Vencedor"].str.lower() == clube
    return (data[filter]['Vencedor'].count()).astype(np.int64)

def getNumberDepartures(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    return (data[filter1]['Clube 1'].count() + data[filter2]['Clube 2'].count()).astype(np.int64)

def getPoints(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    filter3 = df["Vencedor"].str.lower() == clube
    filter4 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter5 = df["Vencedor"].str.lower() == '-'
    
    v1 = data[(filter1) & (filter3)]
    v1 = v1['Vencedor'].count()
    v2 = data[(filter2) & (filter3)]
    v2 = v2['Vencedor'].count() 
    v3 = data[(filter4) & (filter5)]
    v3 = v3['Vencedor'].count()
    return ((v2*3)+(v1*3)+v3).astype(np.int64)

def getDraw(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = df["Vencedor"].str.lower() == '-'
    df = data[(filter1) & (filter2)]
    empates = df['Vencedor'].count()
    return empates.astype(np.int64)

def getDefeats(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = (df["Vencedor"].str.lower() != clube) & (df["Vencedor"].str.lower() != '-')
    df = data[(filter1) & (filter2)]
    derrotas = df['Vencedor'].count()
    return derrotas.astype(np.int64)

def getGP(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1  =  df["Clube 1"].str.lower() == clube
    filter2  =  df["Clube 2"].str.lower() == clube
    df1      =  data[(filter1)]
    df2      =  data[(filter2)]
    placar1  =  df1['Placar'].str.split('x')
    placar2  =  df2['Placar'].str.split('x')
    gp       =  0
    gc       =  0
    for g1, g2 in placar1:
        gp = (gp + pd.to_numeric( g1 ))
        gc = (gc + pd.to_numeric( g2 ))
            
    for g1, g2 in placar2:
        gp = (gp + pd.to_numeric( g2 )) 
        gc = (gc + pd.to_numeric( g1 ))
    return gp, gc

In [13]:
dfTable = cb[["Clubes"]].copy()
for column in ["PG", "J", "V", "E", "D", "GP", "GC", "SG"]:
    dfTable[column] = pd.np.nan
    
    
for index, row in dfTable.iterrows():
    c1  =  row['Clubes']
    if type(c1) != unicode:
        c1 =  c1.decode('utf-8')
    c1 =  c1.strip()
    
    pg       =  getPoints(dfDataChampionship, c1 )
    j        =  getNumberDepartures(dfDataChampionship, c1 )
    v        =  getNumberWinner(dfDataChampionship, c1 )
    e        =  getDraw(dfDataChampionship, c1 )
    d        =  getDefeats(dfDataChampionship, c1 )
    gp, gc   =  getGP(dfDataChampionship, c1 )
    
    if c1 == u'vasco':
        pg  += 2 #Vasco venceu o processo na justica e ganhou os pontos da partida contra o brasiliense
        v   += 1
        e   -= 1
        gp  -= 1
        gc  -= 2
    elif c1 == u'brasiliense':
        pg  -= 1 #Foi punido pela justiça
        e   -= 1
        d   += 1
        gc  -= 1
        gp  -= 2
        
    dfTable.at[index, 'PG'] = pg
    dfTable.at[index, 'J']  = j
    dfTable.at[index, 'V']  = v
    dfTable.at[index, 'E']  = e
    dfTable.at[index, 'D']  = d
    dfTable.at[index, 'GP'] = gp
    dfTable.at[index, 'GC'] = gc
    dfTable.at[index, 'SG'] = gp - gc
    
    #print c1

dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
0,juventude,47.0,38.0,13.0,8.0,17.0,44.0,54.0,-10.0
1,vasco,61.0,38.0,16.0,13.0,9.0,56.0,48.0,8.0
2,são paulo,78.0,38.0,22.0,12.0,4.0,66.0,32.0,34.0
3,grêmio,67.0,38.0,20.0,7.0,11.0,64.0,45.0,19.0
4,botafogo,51.0,38.0,13.0,12.0,13.0,52.0,50.0,2.0
5,são caetano,36.0,38.0,9.0,9.0,20.0,37.0,53.0,-16.0
6,palmeiras,44.0,38.0,12.0,8.0,18.0,58.0,70.0,-12.0
7,santa cruz,28.0,38.0,7.0,7.0,24.0,41.0,76.0,-35.0
8,atlético-pr,48.0,38.0,13.0,9.0,16.0,61.0,62.0,-1.0
9,goiás,55.0,38.0,15.0,10.0,13.0,63.0,49.0,14.0


In [14]:
reorderlist = [ 'São Paulo', 'Internacional', 'Grêmio', 'Santos', 'Paraná', 'Vasco', 'Figueirense', 
                'Goiás', 'Corinthians', 'Cruzeiro', 'Flamengo', 'Botafogo', 'Atlético-pr', 
                'Juventude', 'Fluminense', 'Palmeiras', 'Ponte Preta', 'Fortaleza', 'São Caetano', 'Santa Cruz']
listIndex   = []

#Processa a lista para gerar a lista de ordenação
for clube in reorderlist:
    for index, row in dfTable.iterrows():
        #Verifica se o valor de clube é um unicode, se não, convete isso.
        if type(clube) != unicode:
            c =  clube.decode('utf-8')
        else:
            c =  clube
        
        c1  =  row['Clubes']
        #print c1
        if type(c1) != unicode:
            c1 =  c1.decode('utf-8')
        
        c1 =  c1.strip()
        c  =  c.lower()
        
        #Verifica se o clube iterado é igual ao clube da lista, se sim
        #retorna seu indice
        if c1 == c:
            listIndex.append( index )
            break
            
#Reordena a lista
dfTable  =  dfTable.reindex(index=listIndex, columns=["Clubes", "PG", "J", "V", "E", "D", "GP", "GC", "SG"])
#Com a lista ordenada, corrige os indices.
dfTable.index =  np.arange(1, len(dfTable)+1)
dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
1,são paulo,78.0,38.0,22.0,12.0,4.0,66.0,32.0,34.0
2,internacional,69.0,38.0,20.0,9.0,9.0,52.0,36.0,16.0
3,grêmio,67.0,38.0,20.0,7.0,11.0,64.0,45.0,19.0
4,santos,64.0,38.0,18.0,10.0,10.0,58.0,36.0,22.0
5,paraná,60.0,38.0,18.0,6.0,14.0,56.0,49.0,7.0
6,vasco,61.0,38.0,16.0,13.0,9.0,56.0,48.0,8.0
7,figueirense,57.0,38.0,15.0,12.0,11.0,52.0,44.0,8.0
8,goiás,55.0,38.0,15.0,10.0,13.0,63.0,49.0,14.0
9,corinthians,53.0,38.0,15.0,8.0,15.0,41.0,46.0,-5.0
10,cruzeiro,53.0,38.0,14.0,11.0,13.0,52.0,45.0,7.0


** Estes dados possuem um erro em relação a fonte de onde foi obtido.**
** Por algum motivo na fonte o Vasco tem somente 57 pontos, analisando os dados, isso não confere e preferi manter com os 61 pontos, já que não encontrei um ero, talvez um erro de digitação por parte do pessoal da fonte**